# Packages used

In [1]:
from astropy.io import fits 
import matplotlib.pyplot as plt

import os
import glob
import numpy as np

## All events used are in the **All events** folder:

- ### https://drive.google.com/drive/folders/1YxhCUdBSnPx4sSMn7hs2LoV-k0eC-x3y?usp=drive_link

| ID | AR | DATE | TIME | ROWS | COLUMNS |
|:--:|:------:|:------------:|:--------:|:----:|:---------:|
| 1 | 10930 | 2006/12/12 | 035005 | 512 | 1000 |
| 2 | 11692 | 2013/03/14 | 210005 | 384 | 562 |
| 3 | 11967 | 2014/02/04 | 190005 | 408 | 831 |
| 4 | 12209 | 2014/11/23 | 062123 | 408 | 875 |
| 5 | 12297 | 2015/03/10 | 074505 | 512 | 505 |
| 6 | 12297 | 2015/03/10 | 093005 | 512 | 512 |
| 7 | 12297 | 2015/03/10 | 110406 | 512 | 511 |
| 8 | 12297 | 2015/03/10 | 165005 | 512 | 512 |
| 9 | 12497 | 2016/02/12 | 231005 | 512 | 876 |
| 10 | 12567 | 2016/07/22 | 233504 | 512 | 967 |
| 11 | 12672 | 2017/08/23 | 224005 | 512 | 724 |
| 12 | 12727 | 2018/11/17 | 130006 | 384 | 384 |
| 13 | 12736 | 2019/03/22 | 010205 | 512 | 909 |

In [2]:
#Folder names
arquivos_fits = glob.glob('All events/*.fits')  
nomes_arquivos = [os.path.basename(f) for f in arquivos_fits]

In [3]:
glob.glob('All events/*.fits')

['All events\\combined_10930_20061212_035005.fits',
 'All events\\combined_11692_20130314_210005.fits',
 'All events\\combined_11967_20140204_190005.fits',
 'All events\\combined_12209_20141123_062123.fits',
 'All events\\combined_12297_20150310_074505.fits',
 'All events\\combined_12297_20150310_093005.fits',
 'All events\\combined_12297_20150310_110406.fits',
 'All events\\combined_12297_20150310_165005.fits',
 'All events\\combined_12497_20160212_231005.fits',
 'All events\\combined_12567_20160722_233504.fits',
 'All events\\combined_12672_20170823_224005.fits',
 'All events\\combined_12727_20181117_130006.fits',
 'All events\\combined_12736_20190322_010205.fits']

## Generating a Fits file with all events

In [4]:
# List .fits files in current folder
arquivos_fits = glob.glob('All events/*.fits')

# Initialize list
dados_todos = []

# Loop to process each file
for arquivo in arquivos_fits:
    
    # Open the FITS file
    with fits.open(arquivo) as hdul:    
        
        data_stokes = hdul[0].data
        stokes = np.copy(data_stokes)
        stokes = stokes.astype("float64") 

        # Stokes normalization (I_c varies from author to author)
        I_qs =  np.mean(stokes[:,:, 0, 0:10]) 
        stokes /= I_qs

        # Minimum index (intensity profile)       
        ind_min = 30

        # Selection of 31 points (centered on the minimum)
        I_ = stokes[:,:,0, ind_min-15:ind_min+16]  
        Q_ = stokes[:,:,1, ind_min-15:ind_min+16]
        U_ = stokes[:,:,2, ind_min-15:ind_min+16]
        V_ = stokes[:,:,3, ind_min-15:ind_min+16]

        IQUV_concat = np.concatenate([I_, Q_, U_, V_], axis=-1)

        data_reshaped = IQUV_concat.reshape(-1, 124)

        dados_todos.append(data_reshaped)

# # Now stack all data vertically 
data_array = np.vstack(dados_todos)

# Save as FITS file
hdu = fits.PrimaryHDU(data_array)
hdu.writeto('combined_ALL_31points_mean.fits', overwrite=True) 